# **Hands-on 4: Exemplo Lena-Simple-EPC**

### **Objetivo:**
A proposta deste Hands-on é descrever e explicar o código de exemplo Lena-Simple-EPC.cc e por consequência explicar o que é EPC (*Evolved Packet Core*). Além disso, objetiva-se construir um script de simulação para ser utilizado em uma campanha de convergência de métricas.


### **Cenário:**

O Evolved Packet Core (EPC) é uma estrutura destinada a permitir o uso de Protocolos de Internet (IPs) em aparelhos 4G Long-Term Evolution (LTE) para fornecer comunicação via voz e dados como uma única rede convergente. Neste Hands on veremos como é implementada esta tecnologia em um exemplo do ns3.

## **Requisitos:**

*   Ter instalado o ns-3.34 ou superior; e
*   Ter realizado as leituras preliminares necessárias.


## **Parte 1: Como funciona o Exemplo Lena-Simple-EPC.cc?**

Este *script* realiza a comunicação de dois UEs e uma rede de internet através de dois eNBs conectados ao EPC. O arranjo final é semelhante à ilustração abaixo.

![1.png](FIGS/1.png)

### **Passo 1: Entendendo o código.**

Inicialmente são determinados os includes das bibliotecas a serem utilizadas no exemplo e a descrição do using namespace ns3. 

![2.png](FIGS/2.png)

Após isso, a *main* é iniciada e são definidas as variáveis que serão utilizadas em outras partes do script.

![3.png](FIGS/3.png)

As linhas de código abaixo estabelecem as variáveis que poderão ser alteradas via terminal.

![4.png](FIGS/4.png)

Caso o boleano **useCa** for configurado para “True” a seguinte função if é utilizada:

![5.png](FIGS/5.png)

Primeiramente, além do LteHelper, já apresentado no exemplo Lena-Simple, utilizaremos uma classe EpcHelper adicional, que se encarregará de criar as entidades EPC e a topologia da rede. Neste caso, não é possível utilizar o EpcHelper diretamente, pois é uma classe base abstrata, portanto, é necessário utilizar uma de suas classes filhas, que fornecem diferentes implementações de topologia EPC. 

Neste exemplo, foi considerado o PointToPointEpcHelper, que implementa um EPC baseado em links ponto a ponto.

Após isso, é “avisado” ao **LTE Helper** que o **EPC** será utilizado por meio da linha de código lteHelper->SetepcHelper(epcHelper). Essa etapa é necessária para que o **LTE Helper** acione a configuração **EPC** apropriada em correspondência a alguma configuração importante, como quando um novo eNB ou UE é adicionado ou um EPS bearer é criado. O **EPC Helper** cuidará automaticamente das configurações necessárias, como a criação do link S1 e a configuração do portador S1, sem que haja intervenção do usuário.

É importante salientar que, chamar a linha de código supracitada ativa o uso do **EPC** e tem o efeito colateral de que qualquer novo **LteEnbRrc** criado terá o atributo **EpsBearerToRlcMapping** definido como *RLC_UM_ALWAYS* em vez de RLC_SM_ALWAYS, se o último for o padrão; caso contrário, o atributo não será alterado (por exemplo, se você alterou o padrão para *RLC_AM_ALWAYS*, ele não será alterado).

![6.png](FIGS/6.png)

Deve-se notar que o **EpcHelper** também criará automaticamente o nó **PGW** e o configurará para que possa lidar adequadamente com o tráfego da rede **LTE**. Ainda assim, é necessário utilizar a linha de código abaixo para conectar o **PGW** a outras redes **IPv4/IPv6** (por exemplo, a internet, ou a outro **EPC**).

![7.png](FIGS/7.png)

Com isso, é criado um único RemoteHost.

![8.png](FIGS/8.png)

Assim, pode-se configurar todos os aspectos básicos da conexão e da internet, como o throughput, tamanho do arquivo e atraso do sistema. Depois é feita a conexão do **PGW** com a internet.

![9.png](FIGS/9.png)

Nas seguintes linhas de código, são criadas através do método **NodeContainer** e posicionadas através do método **MobilityModel** as **eNBs** e **UEs** da rede **LTE** estruturada. É interessante notar que os vetores são implementados por apenas uma variável na direção “x” que começa em 0 e é acrescida em 1 a cada iteração do laço **for**, enquanto as direções “y” e “z” são sempre zero.

![10.png](FIGS/10.png)

Em seguida, os *devices* LTE são instalados nos nós criados.

![11.png](FIGS/11.png)

As linhas de código abaixo configuram as UEs para rede IP. Em primeiro lugar, é chamado o método *AssignUeIpv4Adress* para gerenciar os endereços de Ips, após isso, são utilizados duas funções **for**, a primeira é utilizado para implementar os endereços de Ip nas UEs, e o segundo conectará as UEs a seus respectivos eNodeBs.

![12.png](FIGS/12.png)

Após isso, os aplicativos são instalados nos nós UE LTE.

![13.png](FIGS/13.png)

O exemplo ainda nos permite decidir o *status* do downlink, uplink e da comunicação entre os nodes entre ativado ou desativado, modificando as variáveis booleanas implementadas logo após o início da main.

![14.png](FIGS/14.png)

Finalmente, é configurado o tempo de funcionamento das aplicações *server* e *client*, definido o tempo final da simulação - por meio da variável estabelecida no começo do script - e dado o comando de *start* e *destroy* na simulação. 

![15.png](FIGS/15.png)

### **Passo 2: Executando o script.**

Inicialmente, copie o exemplo **lena-simple-epc.cc** para a pasta scratch. Após isso, crie uma pasta chamada **“resultados_lena-simple-epc”** no diretório principal do seu ns-3. Assim, execute a linha de comando abaixo.

```
./waf --cwd="resultados_lena-simple-epc"/ --run lena-simple-epc
```

![16.png](FIGS/16.png)

Assim, são gerados os mesmos tipos de arquivos gerados pelo exemplo **lena-simple.cc.**

![17.png](FIGS/17.png)

Por padrão, a execução do script resulta nos dados do intervalo de tempo entre 0.5 e 1.1 segundos da simulação e o espaçamento entre cada medida é de 0.1 segundos. Esses valores podem ser alterados substituindo os valores das variáveis **simTime** e **interPacketInterval** e alterando o tempo em que as aplicações irão iniciar.

### **Passo 3: Adicionando o REM.**

Copie o exemplo **lena-simple-epc** para a pasta scratch e renomeie-o **lena-simple-epc-rem**, após isso, introduza as linhas de código abaixo

```
Ptr<RadioEnvironmentMapHelper> remHelper = CreateObject<RadioEnvironmentMapHelper> ();
remHelper->SetAttribute ("Channel", PointerValue (lteHelper->GetDownlinkSpectrumChannel ()));
remHelper->SetAttribute ("OutputFile", StringValue ("rem.out"));
remHelper->SetAttribute ("XMin", DoubleValue (-400.0));
remHelper->SetAttribute ("XMax", DoubleValue (400.0));
remHelper->SetAttribute ("YMin", DoubleValue (-300.0));
remHelper->SetAttribute ("YMax", DoubleValue (300.0));
remHelper->SetAttribute ("Z", DoubleValue (0.0));
remHelper->Install ();

```
logo antes das linhas de código

```
Simulator::Stop (simTime);
Simulator::Run ();
```
Após isso, comente a seguinte linha de código 
```
lteHelper->EnableTraces ();
```
de forma que ela fique como mostrado abaixo.

![18.png](FIGS/18.png)

Fazemos isso pois a ativação do REM pode gerar traces incompletas e não precisaremos destes arquivos no momento. Após isso, crie uma nova pasta chamada **“resultados_lena-simple-epc-rem”** abra o terminal na pasta principal do ns-3 e execute a linha de comando

```
./waf --cwd="resultados_lena-simple-epc-rem"/ --run lena-simple-epc-rem
```
Você perceberá que apenas o arquivo rem.out foi gerado na pasta.

![19.png](FIGS/19.png)

Após abrir o terminal na pasta onde está o arquivo rem.out, abra o gnuplot e, então, insira os comandos abaixo.


```
set view map;
set term x11;
set xlabel "X"
set ylabel "Y"
set cblabel "SINR (dB)"
plot "rem.out" using ($1):($2):(10*log10($4)) with image
```
A figura obtida deste passo será como a mostrada abaixo. 

![20.png](FIGS/20.png)

### **Passo 4: Extrair as informações apenas do downlink.**

A análise que se deseja realizar neste hands-on é sobre o *throughput* do *downlink* dos UEs estabelecidos. Para isso, serão analisados os dados presentes no arquivo DlTxPhyStats. Visto isso, não é necessário obter os arquivos contendo informação de *uplink* e comunicação entre os UEs. Para isso, apague os arquivos da pasta **“resultados_lena-simple-epc”** e altere as variáveis **disableUp** e **disablePl** pelo terminal quando for realizar a execução do script usando a linha de comando abaixo.

```
./waf --cwd="resultados_lena-simple-epc"/ --run "lena-simple-epc --disableUl=true --disablePl=true"
```
![21.png](FIGS/21.png)

## **Parte 2: Modificando o exemplo para uma campanha.**

### **Passo 1: Atualizar as variáveis do script.**

Para realizar uma campanha com base neste script é necessário que todas as variáveis que se deseja modificar para gerar diferentes cenários nas simulações estejam declaradas e com suas modificações habilitadas via linha de comando. 

Para isso, substitua os *includes* e as listas de variáveis do código original por estas [linhas de código](https://github.com/vicentesousa/IC_LTE_ns3/blob/main/fase_01/HD_04/variaveis).

Todas essas variáveis serão posteriormente utilizadas em outras partes do script. Note que algumas variáveis “já existentes” foram alteradas a fim de simplificar a sua utilização no cenário que está sendo construído, o que necessitará que se **modifique o nome das variáveis presentes em outros locais do código**, **o endereço da pasta do computador** e **altere algumas das lógicas** que utilizavam as mesmas.

### **Passo 2: Adicionar *Fading* e *Shadowing*.**

O *fading* é adicionado da mesma forma que foi visto anteriormente no hands-on do exemplo **lena-fading.cc**, utilizando *fading traces*. Então, caso o valor da variável **useFading** for verdadeiro, as linhas de código com as configurações do fading devem ser ativadas. Repare que agora existe uma variável chamada **fadingEnd** que contém o endereço dos *fading traces* que devem ser utilizados na simulação.

Já o *shadowing* é um atributo do modelo de perda de propagação ns3::ThreeGppUmaPropagationLossModel, desta forma, é necessário, antes de tudo, adicionar esse modelo ao script. Após isso, como o shadowing é **nativamente ativo**, caso o valor da variável **useShadowing** for falso, é necessário configurar o atributo **"ShadowingEnabled"** como falso. A imagem abaixo apresenta as linhas de código que precisam ser adicionadas e onde as mesmas devem ser situadas.

![22.png](FIGS/22.png)

### **Passo 3: Adicionar diferentes EPS Bearers.**

Quando utilizamos EPC a ativação de bearers é feita de uma maneira ligeiramente diferente em relação ao que foi feito para uma simulação somente LTE. Primeiro, o método ActivateDataRadioBearer não deve ser usado quando o EPC é usado. Além disso, quando o EPC é usado, o portador EPS padrão será ativado automaticamente quando você chamar LteHelper::Attach (). Todavia, se você deseja configurar um EPS bearer dedicado, pode fazê-lo usando o método LteHelper::ActivateDedicatedEpsBearer (). Este método toma como parâmetro o Traffic Flow Template (TFT), que é uma estrutura que identifica o tipo de tráfego que será mapeado para o EPS bearer dedicado. Desta forma, adicione as linhas de código abaixo logo antes das linhas de código que definem a aplicação.

```
Ptr<EpcTft> tft = Create<EpcTft> ();
EpcTft::PacketFilter pf;
pf.localPortStart = 1100;
pf.localPortEnd = 1100;
tft->Add (pf);
lteHelper->ActivateDedicatedEpsBearer (ueLteDevs,
                                       EpsBearer (EpsBearer::NGBR_VIDEO_TCP_DEFAULT),
                                       tft);
```
![23.png](FIGS/23.png)

### **Passo 4: Adicionar tráfego de Poisson**

Buscando utilizar um tráfego com um comportamento mais aproximado do que acontece na prática, pensou-se em adotar um tráfego que utiliza uma distribuição de Poisson. Para isso, utiliza-se um helper de aplicação que suporte o uso de variáveis aleatórias para reger o intervalo de criação dos pacotes. Assim, utiliza-se o OnOffHelper, configurando-o de modo que o tempo em Off é exponencial e o tempo On é longo o bastante para enviar um pacote. 

As linhas de código que definiam a aplicação no script original devem ser modificadas por estas [linhas de código](https://github.com/vicentesousa/IC_LTE_ns3/blob/main/fase_01/HD_04/poisson).

Repare que existe uma lógica envolvendo as variáveis **useUdp** e **usePoisson** para que apenas um dos tipos de tráfego possa ser utilizado por vez. Além disso, algumas variáveis foram modificadas e outras adicionadas - como as variáveis **lambda, appStartTime, simTime** - para que se pudesse ter maior controle da simulação.  

### **Passo 5: Adicionar TraceSentPacker.**

Para saber o exato momento em que o dado foi transmitido pela aplicação, sem os atrasos causados por outras camadas e, em outro momento, ter a possibilidade de averiguar se essa transmissão está realmente seguindo uma distribuição de Poisson, estabelecemos um *callback* que conecta o *trace sink* do Tx do OnOffHelper com a função TranceSentPacket, que gera um arquivo de mesmo nome contendo cada dado de tempo em que um dado foi enviado pela aplicação. Sabendo disso, insira as linhas de código abaixo **após os *includes*** e **acima da *main*** do *script*.

```
NS_LOG_COMPONENT_DEFINE("LenaSimpleEpc");
std::ofstream m_TxTraceFile;
std::string m_TxTraceFileName;

void TraceSentPacket(std::string context, Ptr<const Packet> m_txTrace) {
//Vector position = model->GetPosition ();
//NS_LOG_UNCOND (context << "Time Tx: "
//              << Simulator::Now ().GetSeconds ()
//              << " Packet = " << m_txTrace);
	if (!m_TxTraceFile.is_open()) {
		m_TxTraceFileName = "TxSentTrace.txt";
		m_TxTraceFile.open(m_TxTraceFileName.c_str());
		m_TxTraceFile << "Time" << std::endl;

		if (!m_TxTraceFile.is_open()) {
			NS_FATAL_ERROR("Could not open tracefile");
		}
	}

	m_TxTraceFile << Simulator::Now() << std::endl;
}
// To be used in tic toc time counter
clock_t startTimer;
time_t beginTimer;

// Implementation of tic, i.e., start time counter
void tic() {
	beginTimer = time(&beginTimer);
	struct tm *timeinfo;
	timeinfo = localtime(&beginTimer);
	std::cout << "simulation start at: " << asctime(timeinfo) << std::endl;
}
// implementation of toc, i.e., stop time counter
    double toc() {
	time_t finishTimer = time(&finishTimer);
	double simTime = difftime(finishTimer, beginTimer) / 60.0;
	struct tm *timeinfo;
	timeinfo = localtime(&finishTimer);
	std::cout << "simulation finished at: " << asctime(timeinfo) << std::endl;
	//
	std::cout << "Time elapsed: " << simTime << " minutes" << std::endl;
	//
	return simTime;
}
```
Após isso, insira as seguintes linhas de código após as definições de start e stop time da aplicação com tráfego de Poisson, como mostra a figura abaixo.
```
std::ostringstream oss;
		oss << "/NodeList/"
			<< remoteHost->GetId()
			<< "/ApplicationList/0"
			<< "/$ns3::OnOffApplication/Tx";
Config::Connect(oss.str(), MakeCallback(&TraceSentPacket));
```
![24.png](FIGS/24.png)


### **Passo 6: Adicionar Flow Monitor**

Depois de termos obtido as métricas do traces fornecidos pelo próprio **LteHelper** e pelo *callback* que construímos, podemos obter ainda mais parâmetros do cenário construído utilizando o Flow Monitor, que é um *helper* que é configurado para este fim. Para adicioná-lo ao script, começa-se inserindo as linhas de código abaixo logo antes do **Simulator::Stop()**, como mostra a figura abaixo.
```
FlowMonitorHelper flowmonHelper;
NodeContainer endpointNodes;
endpointNodes.Add(remoteHost);
endpointNodes.Add(ueNodes);

Ptr<ns3::FlowMonitor> monitor = flowmonHelper.Install(endpointNodes);
monitor->SetAttribute("DelayBinWidth", DoubleValue(0.001));
monitor->SetAttribute("JitterBinWidth", DoubleValue(0.001));
monitor->SetAttribute("PacketSizeBinWidth", DoubleValue(20));
AsciiTraceHelper asciiTraceHelper;
```
![25.png](FIGS/25.png)

Após isso, é necessário configurar o Flow Monitor. Para isso, insira essas [linhas de código](https://github.com/vicentesousa/IC_LTE_ns3/blob/main/fase_01/HD_04/flow%20monitor) após o **Simulator::Run()**.

O resultado será a criação de dois arquivos, um para *downlink* e outro para *uplink* cujos nomes serão determinados pelo tempo de simulação, número de UEs e número de eNBs.


### **Passo 7: Executar o script novamente.**

Após ter concluído todos os passos anteriores, execute mais uma vez o *script* utilizando a linha de comando abaixo.

```
./waf --cwd="resultados_lena-simple-epc"/ --run "lena-simple-epc --disableUl=true --disablePl=true"
```
Os arquivos gerados devem ser os mostrados na figura abaixo.

![26.png](FIGS/26.png)

Repare que, por se tratar de uma simulação envolvendo apenas o *downlink*, o único arquivo gerado pelo **Flow Monitor** contendo informações será o desse enlace. A formatação desse arquivo é mostrada na figura abaixo.

![27.png](FIGS/27.png)